In [5]:
import random
from bisect import bisect_left


#Generate N elements for each layer and select M out of N
class Element: #define a element
 def __init__(self, eID, layer, monitored):
    self.id = eID
    self.layer = layer
    self.monitored = monitored

 def infoOut(self):
    print("Element info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", monitored = " 
          + str(self.monitored) 
         ) 

class EvtQueue:
 def __init__(self):
    self.evtID = 0 #initial number of events
    self.queue = [] #empty queue
                
 def getNextEvt(self): #obtain the head of line fault in the evt queue
    if (len(self.queue) > 0):
        currFault = self.queue[0]
        self.queue.remove(currFault) #delete the currFault 
        return currFault 
    else:
        return None
    
 def hasNextEvt(self): #check if the queue is not empty
    if (len(self.queue) > 0):
        return True
    else:
        return False
        
 def getEvtNum(self):
    return self.evtID
    
 def infoOut(self):
    for f in self.queue:
        f.infoOut()
    
 def insertEvt(self, newFault):
    def insert(seq, keys, item, keyfunc=lambda v: v):
        """Insert an item into a sorted list using a separate corresponding
       sorted keys list and a keyfunc() to extract the key from each item.
       Based on insert() method in SortedCollection recipe:
       http://code.activestate.com/recipes/577197-sortedcollection/
        """
        k = keyfunc(item)  # Get key.
        i = bisect_left(keys, k)  # Determine where to insert item.
        keys.insert(i, k)  # Insert key of item to keys list.
        seq.insert(i, item)  # Insert the item itself in the corresponding place.
    keys = [fault.evtTime for fault in self.queue]
    insert(self.queue, keys, newFault, keyfunc=lambda x: x.evtTime) #insert on the order of evtTime to the queue
    self.evtID = self.evtID + 1
    
#Generate a seqence of faults sorted in an increasing order of evtTime
class Fault: #define a fault
 def __init__(self, fID, initlayer, layer, initTime, evtTime, assocElement, impactVector):
    self.id = fID
    self.initLayer = initlayer
    self.layer = layer
    self.initTime = initTime #the inital time when the fault is generated for calculating delay
    self.evtTime = evtTime #the time when the fault is re-inserted in the event queue
    self.assocElement = assocElement #within N elements
    self.impactVector = impactVector #impact vector of this fault
    
 def infoOut(self):
    print("Fault info: id = "
          + str(self.id)
          + ", initLayer = " 
          + str(self.initLayer) 
          + ", layer = " 
          + str(self.layer) 
          + ", initTime = " 
          + str(self.initTime) 
          + ", evtTime = " 
          + str(self.evtTime) 
          + ", assocElementID = " 
          + str(self.assocElement.id)
          + ", monitored = " 
          + str(self.assocElement.monitored)
          + ", impactVector = " 
          + str(self.impactVector)          
         )  
    
#This class manages all elements init, update in the system
class Elements:
    def __init__(self, N, M_N):
        self.M_N = M_N
        self.M = []
        self.N = N
        
        for i in range(len(N)):
            self.M.append(self.N[i]*self.M_N)
    
        self.elements = []
        elementID = 0
        for i in range(len(N)): #for each layer i
            j = 0
            elements_layer = []
            while j < self.N[i]: #generate Ni elements
                elementID = elementID + 1
                elements_layer.append(Element(elementID, i, False))
                j = j + 1
            count = 0
            assert(self.M[i] <= self.N[i]), "Total elements Ni is too small to reach Mi!"
            while count < self.M[i]: #select Mi elements out of Ni 
                randomElement = random.randrange(N[i])
                #print(randomElement)
                if elements_layer[randomElement].monitored == False:
                    elements_layer[randomElement].monitored = True
                    count = count + 1
            self.elements.append(elements_layer)
    
    #Update elements based on a given M_N, randomly select M out of N, but no change of the array 
    def update_elements(self, M_N):
        self.M_N = M_N
        #print(self.M)
        #print (self.M_N)
        #print(self.N)
        for i in range(len(self.M)):
            self.M[i] = self.N[i]*self.M_N
        #print(self.M)
        
        for i in range (len(self.N)): #for each layer i
            j = 0
            elements_layer = self.elements[i]
            while j < self.N[i]: #generate Ni elements
                #elementID = elementID + 1
                elements_layer[j].monitored = False #reset all to False
                j = j + 1
            
            count = 0
            assert(self.M[i] <= self.N[i]), "Total elements Ni is too small to reach Mi!"
            while count < self.M[i]: #select Mi elements out of Ni 
                randomElement = random.randrange(self.N[i])
                #print(randomElement)
                if elements_layer[randomElement].monitored == False:
                    elements_layer[randomElement].monitored = True
                    count = count + 1
        
    def infoOut(self):
        for i in range (len(self.N)): #for each layer i
            j = 0
            elements_layer = self.elements[i]
            strOut = ""
            while j < self.N[i]: #generate Ni elements
                #elementID = elementID + 1
                strOut = strOut + str(elements_layer[j].monitored) + " " 
                j = j + 1
            strOut = strOut + "\n"
            print(strOut)
        
def One_Experiment(_option, _failureRatio, _M_N, _accuracyRCA, _accuracyCorrelation, _accuracyFaultDetection, _fixMN, _lastTime, _THRESHOLD_DOWN, _THRESHOLD_UP):
 def generate_impactVector(numLayer): #generate an impact vector with a random impact value at element j at layer i, where element j is randomly selected among the Ni number of elements of layer i
    vectors = []
    #if (i in range (0, Ni): #assign all zeros to the ImpactVector
    #    vector[i] = 0
    for i in range(numLayer):
        vector = []        
        vector.append(i) #vector[0] = i, the layer i
        vector.append(random.randint(0, N[i]-1)) #vector[1] = j, randomly select the element j
        vector.append(random.random()) #vector[2] = the impact value
        vectors.append(vector)
    return vectors
                   
 def generate_newFault(evtTime, elements): #generate a new fault at evtTime among a set of elements
    layer = random.randint(0, I-1) #randomly select a layer, 0 ~ I-1
    elements_layer = elements.elements[layer] #find corresponding elements_layer 
    elementIndex = random.randint(0, len(elements_layer)-1) #randomly (uniformaly) assign fault to Ni elements a..t layer i
    newFault = Fault(faultsTotal, layer, layer, evtTime, evtTime, elements_layer[elementIndex], generate_impactVector(I))  
    return newFault

 def generate_impactedFault(vector, elements): #generate a impacted fault at evtTime among a set of elements based on an impact vector at layer i
    DELTA = random.uniform(1, DELTA_RANGE)
    nextEvtTime = currFault.evtTime + DELTA
    #for the new impacted fault, the impact vector is None
    newFault = Fault(faultsTotal, currFault.initLayer, vector[0], currFault.initTime, nextEvtTime, 
                     elements.elements[vector[0]][vector[1]], None)  
    return newFault

 """ 
 Initialization
 """
 #Input loop variables
 option = _option
 failureRatio = _failureRatio
 M_N = _M_N    
 accuracyFaultDetection = _accuracyFaultDetection
 accuracyCorrelation = _accuracyCorrelation
 accuracyRCA = _accuracyRCA
 LASTTIME = _lastTime; #simulation running time
 THRESHOLD_DOWN = _THRESHOLD_DOWN 
 THRESHOLD_UP = _THRESHOLD_UP #0.9 #M_N*0.8 #Threshold # C_R/faultsTotal = M/N * accuracyFaultDetection * accuracyRCA 

 #System architecture
 I = 5 # number of layers 0 ~ (I-1)
 N = [100, 100, 100, 100, 100] #number of elements for layer i
    
 #Simulation configuration
 SEED = 1
 random.seed(SEED)
 #Algorithm configuration
 THRESHOLD_IMPACTVECTOR = 0.5
 DELTA_RANGE = 10 #the time range (0, DELTA_RANGE) for a fault to be generated by impact factor 
 D_0 = 12 # the largest delay
 D_1 = 9 # a large delay
 D_2 = 6
 D_3 = 3 
 #accuracyFaultDetection = 0.9
 M_N_Change_Rate = 0.05
 THRESHOLD_FAULTS = 0.025 * LASTTIME #check and update M_N frequence relative to the overall system time

 #Counter for monitored/detected/diagnosed faults: 
 C_M = 0 #monitored 
 C_D = 0 #detected
 C_R = 0 #diagnosed
 faultsTotal = 0 #faults != events in queue, one fault may trigger multiple events at different layers for option 4
 delayTotal = 0
 impactedFaultsTotal = 0
 timesThresholdFaults = 1
    
 #Output
 M_N_history = [] #the change of M_N over time for monitoring policy
 timePoint = []
 timePoint.append(0)
 timePoint.append(0)
 timePoint.append(M_N)
 timePoint.append(0)
 M_N_history.append(timePoint)

 #Start simuation
 #initialize elements monitored and not-monitored at each layer
 elements = Elements(N, M_N)
 #elements.infoOut()
 queue = EvtQueue()

 # insert an initial fault with evtTime = 0 in the queue
 evtTime = 0
 newFault = generate_newFault(evtTime, elements)
 queue.insertEvt(newFault)
 faultsTotal = faultsTotal + 1
 #queue.infoOut()


 #start to process a Head of Line fault in the event queue
 while (queue.hasNextEvt()):
    currFault = queue.getNextEvt()
    assert(currFault != None), "Empty queue!"
    #currFault.infoOut()
   
    ### 
    #Fault generation
    ###
    #New fault generation  
    ### 
    #insert the next new fault based on the info of the currFault
    #this new fault generaton is to follow the poission process of new fault generation
    #it is not related to the fault generated due to impact vector
    #only generate a new fault for the current fault with its initTime == evtTime
    if currFault.evtTime == currFault.initTime:
        nextEvtTime = currFault.evtTime + random.expovariate(failureRatio)
        if nextEvtTime <= LASTTIME:
            elementIndex = random.randint(0, len(elements.elements[currFault.layer])-1) #generate a new fault with the same layer as the current fault
            newFault = generate_newFault(nextEvtTime, elements)
            queue.insertEvt(newFault)
            faultsTotal = faultsTotal + 1
            #newFault.infoOut()
    elif currFault.evtTime < currFault.initTime:
        print ("currFault.evtTime < currFault.initTime: Wrong evtTime!")
        
    ### 
    #Monitoring and process the currFault
    ###        
    if option == 1:
        #Calculate counters
        if currFault.assocElement.monitored == True:
            delayTotal = delayTotal + D_1
            C_M = C_M + 1
        else:
            delayTotal = delayTotal + D_0
        #currFault.infoOut()
    elif option == 2 or option == 3 or option == 4:
        #Calculate counters
        if currFault.assocElement.monitored == True:
            accuracy_fault_detection = random.random()
            accuracy_rca = random.random()
            if (accuracy_fault_detection < accuracyFaultDetection): 
                if (accuracy_rca < accuracyRCA):
                    delayTotal  = delayTotal + D_3
                    C_R = C_R + 1
                else:   
                    delayTotal = delayTotal + D_2
                    C_D = C_D + 1
            else:
                delayTotal = delayTotal + D_1
                C_M = C_M + 1
        else:
            delayTotal = delayTotal + D_0
        #currFault.infoOut()
    
    ### 
    #Impacted fault generation
    ### 
    #generate impacted fault from the curFault based on the impact vector
    #Impacted fault does not have impact vector further generated since we only conside one round of impact across layer 
    #if option == 1 or option == 2 or option == 3:
    #Important: Option 4 if the element is not monitored, impact fault will still be generated without cross-layer analysis
    if option == 1 or option == 2 or option == 3 or (option == 4 and currFault.assocElement.monitored == False):
        if (currFault.impactVector != None): #not for the impacted faults with None impact vector
            for i in range(len(currFault.impactVector)):
                vector = currFault.impactVector[i]
                if (vector[2] >= THRESHOLD_IMPACTVECTOR):
                    newFault = generate_impactedFault(vector, elements)
                    queue.insertEvt(newFault)
                    faultsTotal = faultsTotal + 1
                    impactedFaultsTotal = impactedFaultsTotal + 1
    
    ###         
    #Impact fault analysis and generation 
    ### 
    #Important: Option 4 to have cross-layer analysis, the fault must be monitored first
    if option == 4 and currFault.assocElement.monitored == True:
        if (currFault.impactVector != None): 
            for i in range(len(currFault.impactVector)):
                vector = currFault.impactVector[i]
                if (vector[2] >= THRESHOLD_IMPACTVECTOR):
                    insert = True
                    newFault = generate_impactedFault(vector, elements)
                    accuracy_correlation = random.random() #randomly generate accuracy of correlation
                    accuracy_rca = random.random() #randomly generate accuracy of RCA
                    if (accuracy_correlation < accuracyCorrelation): #similar to the step of fault detected?
                        insert = False
                        if (accuracy_rca < accuracyRCA):
                            delayTotal = delayTotal + D_3
                            C_R = C_R + 1
                        else: 
                            delayTotal = delayTotal + D_2
                            C_D = C_D + 1
                    if insert == True: #insert impacted fault only if the corellation analysis fails
                        #faultsTotal = faultsTotal + 1  #double check
                        queue.insertEvt(newFault)                        
                        impactedFaultsTotal = impactedFaultsTotal + 1 #double check
                    faultsTotal = faultsTotal + 1  #double check
    
    #update monitoring policy
    if option == 3 or option == 4:
        #with Fixed M/N
        #print("Fixed M/N")
        if _fixMN:
            THRESHOLD_UP = 20
            THRESHOLD_DOWN = 0   
        if currFault.evtTime > timesThresholdFaults*THRESHOLD_FAULTS:
            newM_N = 0
            adjusted = False
            #print("delayTotal/faultsTotal = " + str(delayTotal/faultsTotal) + 
            #      ", THRESHOLD_UP = " + str(THRESHOLD_UP) + 
            #      ", THRESHOLD_DOWN = " + str(THRESHOLD_DOWN))
            if delayTotal/faultsTotal > THRESHOLD_UP: #delay is too long, need to be shorter
                #direct adjust
                #infoOut = "Increase M/N from " + str(M_N) + " to "
                newM_N = min (1, M_N + M_N_Change_Rate) #M_N * (1 + M_N_Change_Rate)) #THRESHOLD_DOWN/(accuracyFaultDetection*accuracyRCA)
                adjusted = True
            elif delayTotal/faultsTotal < THRESHOLD_DOWN: #delay is too good (short can be longer)
                #infoOut = "Decrease M/N from " + str(M_N) + " to "
                newM_N = max(0, M_N - M_N_Change_Rate) #M_N * (1 - M_N_Change_Rate)) #THRESHOLD_UP/(accuracyFaultDetection*accuracyRCA)
                adjusted = True
            if (adjusted):
                M_N = newM_N
                elements.update_elements(newM_N)
                #output
                #infoOut = infoOut + str(newM_N)
                #print(infoOut)
                
            
            timePoint = []
            timePoint.append(timesThresholdFaults)
            timePoint.append(C_R/faultsTotal)
            timePoint.append(M_N)
            timePoint.append(delayTotal/faultsTotal)
            M_N_history.append(timePoint)
            timesThresholdFaults = timesThresholdFaults + 1
 
    #update monitoring policy
    #if option == 3 or option == 4:
    if option == 5 or option == 6:
        #with Fixed M/N
        #print("Fixed M/N")
        if _fixMN:
            THRESHOLD_UP = 1
            THRESHOLD_DOWN = 0   
        if currFault.evtTime > timesThresholdFaults*THRESHOLD_FAULTS:
            #print(str(faultsTotal) + ", " + str(timesThresholdFaults*THRESHOLD_FAULTS))
            #print("C_R/faultsTotal = " + str(C_R/faultsTotal) + 
            #      ", THRESHOLD_UP = " + str(THRESHOLD_UP) + 
            #      ", THRESHOLD_DOWN = " + str(THRESHOLD_DOWN))
            if C_R/faultsTotal > THRESHOLD_UP: 
                #too high, can decrease M_N and accuracy
                #update accuracy for detection and rca
                #decrease M_N
                #infoOut = "Decrease M/N from " + str(M_N) + " to "
                newM_N = max(0, M_N * (1 - M_N_Change_Rate))
                M_N = newM_N
                elements.update_elements(newM_N)
                
                #output
                #infoOut = infoOut + str(newM_N)
                #print(infoOut)
                #elements.infoOut()
                timePoint = []
                timePoint.append(timesThresholdFaults)
                timePoint.append(C_R/faultsTotal)
                timePoint.append(M_N)
                timePoint.append(delayTotal/faultsTotal)
                M_N_history.append(timePoint)
                
            elif C_R/faultsTotal < THRESHOLD_DOWN: 
                #too low, need to increase M_N and accuracy
                #update accuracy for detection and rca
                #increase M_N
                #infoOut = "Increase M/N from " + str(M_N) + " to "
                newM_N = min (1, M_N * (1 + M_N_Change_Rate))
                M_N = newM_N
                elements.update_elements(newM_N)
                
                #output
                #infoOut = infoOut + str(newM_N)
                #print(infoOut)
                #elements.infoOut()    
                timePoint = []
                timePoint.append(timesThresholdFaults)
                timePoint.append(C_R/faultsTotal)
                timePoint.append(M_N)
                timePoint.append(delayTotal/faultsTotal)
                M_N_history.append(timePoint)
                
            else:
                #output
                timePoint = []
                timePoint.append(timesThresholdFaults)
                timePoint.append(C_R/faultsTotal)
                timePoint.append(M_N)
                timePoint.append(delayTotal/faultsTotal)
                M_N_history.append(timePoint)
            
            timesThresholdFaults = timesThresholdFaults + 1
            
 """       
 #Performance calculation    
 print()
 print("option = " + str(option))
 print("Total num of faults: " + str(faultsTotal) + ", total num of impacted faults = " + str(impactedFaultsTotal))
 print("C_M = " + str(C_M) + ", the ratio of monitored, but not detected faults = " + str(C_M/faultsTotal))
 #print("C_D = " + str(C_D) + ", the ratio of detected faults, but not diagnosed = " + str(C_D/faultsTotal))
 print("C_R = " + str(C_R) + ", the ratio of diagnosed faults = " + str(C_R/faultsTotal))
 print("FailureRate = " + str(failureRatio) + ", FaultRecoverDelay = " + str(delayTotal/faultsTotal))
 print()
 print("THRESHOLD_DOWN = " + str(THRESHOLD_DOWN) + " " + "THRESHOLD_UP = " + str(THRESHOLD_UP))
 print("M_N changing curve: # of diagnosedFaults, diagnosedFaults/totalFaults, M_N, totalDelay")
 for point in M_N_history:
    print (str(point[0]) + ", " + str(point[1]) + ", " + str(point[2]) + ", " + str(point[3]))
 """

 #output
 if option == 1 or option == 2:
        dataPoint = []
        dataPoint.append(M_N)
        dataPoint.append(accuracyRCA)
        dataPoint.append(accuracyFaultDetection)
        dataPoint.append(delayTotal/faultsTotal)
        return dataPoint

 if option == 3 or option == 4:
        return M_N_history



In [8]:
LAMDAS = [0.01, 0.08, 0.3] #the fault generation rates for each experiment for each layer
M_NS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #ratio of M/N
accuracyRCAs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] 
accuracyFaultDetections = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] 
accuracyCorrelations = [0.1, 0.3, 0.5, 0.7, 1] 
lastTime = 1000000
THRESHOLD_DOWN = 6
THRESHOLD_UP = 10

In [3]:
option = 1 #options

dataPoints = []
for M_N in M_NS:
    dataPoint = One_Experiment(option, LAMDAS[0], M_N, 0, 0, 0, False, lastTime, THRESHOLD_DOWN, THRESHOLD_UP)
    dataPoints.append(dataPoint)
    
infoOut = ""
for dataPoint in dataPoints:
    infoOut = infoOut + str(dataPoint[0]) + ", "
infoOut = infoOut + "\n"
for dataPoint in dataPoints:
    infoOut = infoOut + str(dataPoint[3]) + ", "
print(infoOut)

0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
11.697843618513325, 11.406907433380084, 11.098983585489806, 10.7895321312721, 10.49701230228471, 10.206479859894921, 9.891417025388737, 9.59504132231405, 9.309674027339643, 9.0, 


In [11]:
option = 2
M_NS = [0.1, 0.5, 1] #ratio of M/N

dataCurves = [] # multiple points for a curve
for M_N in M_NS:
    dataCurve = []
    for accuracyRCA in accuracyRCAs:
        for accuracyFaultDetection in accuracyFaultDetections:
            dataPoint = One_Experiment(option, LAMDAS[0], M_N, accuracyRCA, 0, accuracyFaultDetection, False, lastTime, THRESHOLD_DOWN, THRESHOLD_UP)
            dataCurve.append(dataPoint)
    dataCurves.append(dataCurve)
  
infoOut = ""
for dataCurve in dataCurves:
    infoOut = infoOut + "M_N = " + str(dataCurve[0][0]) + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
    
print(infoOut)

M_N = 0.1
0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
11.665161234646032, 11.63262

In [35]:
option = 3
M_N = 0.5
accuracyRCA = 0.95
accuracyFaultDetection = 0.95
dataCurvesA = []
dataCurvesB = []
dataCurvesC = []
lastTime = 1000000


#fixed M/N
dataCurve = One_Experiment(option, LAMDAS[2], M_N, accuracyRCA, 0, accuracyFaultDetection, True, lastTime, THRESHOLD_DOWN, THRESHOLD_UP)
dataCurvesA.append(dataCurve)

#flexible M/N, THRESHOLD_DOWN = 0.6
M_N = 0.5
THRESHOLD_DOWN = 6
THRESHOLD_UP = 7
dataCurve = One_Experiment(option, LAMDAS[2], M_N, accuracyRCA, 0, accuracyFaultDetection, False, lastTime, THRESHOLD_DOWN, THRESHOLD_UP)
dataCurvesB.append(dataCurve)


#flexible M/N, THRESHOLD_DOWN = 0.8
M_N = 0.8
THRESHOLD_Down = 6 
THRESHOLD_UP = 7
dataCurve = One_Experiment(option, LAMDAS[0], M_N, accuracyRCA, 0, accuracyFaultDetection, False, lastTime, THRESHOLD_DOWN, THRESHOLD_UP)
dataCurvesC.append(dataCurve)

countPoints = min(len(dataCurvesA), len(dataCurvesB), len(dataCurvesC)) 
#countPoints = len(dataCurvesC)


infoOut = ""
for i in range(countPoints):
    dataCurve = dataCurvesA[i]
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[0]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
print(infoOut)


infoOut = ""
for i in range(countPoints):
    dataCurve = dataCurvesB[i]
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[0]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
print(infoOut)

infoOut = ""
for i in range(countPoints):
    dataCurve = dataCurvesC[i]
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[0]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
    #for dataPoint in dataCurve:
    #    print("dataPoint[0] = " + str(dataPoint[0]) + ", dataPoint[1] = " + str(dataPoint[1]) + ", dataPoint[2] = " + str(dataPoint[2]) + ", dataPoint[3] = " + str(dataPoint[3]))

print(infoOut)



0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
0, 0.44897570634376666, 0.44930321960595865, 0.4507748870222381, 0.45070938038777536, 0.4516573947080489, 0.45053349692620975, 0.4508937902205599, 0.45082144765407256, 0.4510661913808192, 0.45103850731370215, 0.45076953674271897, 0.45084185207456406, 0.45089029293509475, 0.4506313217488636, 0.4506120451713038, 0.4503736167101179, 0.45055774101417256, 0.4503549180901189, 0.4504044479235664, 0.4507308943492504, 0.4511949260595229, 0.4512995573317791, 0.45131880486729037, 0.4512389764210948, 0.45117840497980727, 0.4511537609954442, 0.4511603606402282, 0.45112799347928734, 0.4510722778140317, 0.4511598636305387, 0.45120002750920146, 0.45128091667885906, 0.4512982443186669, 0.45108379828218054, 0.4511085782399784, 0.45115952510859314, 0.4512012599332976, 0.4512237415278276, 0.4512723287542782, 0.45114617383287237, 
0.5, 0.5, 0.5, 0.5, 0.5

'\n                timePoint = []\n                timePoint.append(timesThresholdFaults)\n                timePoint.append(C_R/faultsTotal)\n                timePoint.append(M_N)\n                timePoint.append(delayTotal/faultsTotal)\n                M_N_history.append(timePoint)\n'

In [7]:
option = 4
M_N = 0.8
accuracyRCA = 0.95
accuracyFaultDetection = 0.95
THRESHOLD_Down = 6
THRESHOLD_UP = 7
lastTime = 1000000

dataCurves = [] 
for accuracyCorrelation in accuracyCorrelations:
    dataCurve = One_Experiment(option, LAMDAS[2], M_N, accuracyRCA, accuracyCorrelation, accuracyFaultDetection, False, lastTime, THRESHOLD_DOWN, THRESHOLD_UP)
    dataCurves.append(dataCurve)
  
infoOut = ""
for dataCurve in dataCurves:
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[0]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
    
print(infoOut)


0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
0, 0.7879853546771407, 0.7687687116494403, 0.7495701002845708, 0.7297006137328069, 0.7097682532150722, 0.6888806836030522, 0.6678628759544006, 0.6461389450099877, 0.623883666225291, 0.6059150981253578, 0.5915062530111607, 0.5792370091356931, 0.568756536390467, 0.5600451559265116, 0.5523456469637426, 0.5455832151285622, 0.5397743656648889, 0.5346501319854163, 0.5299838418583944, 0.5281516498299088, 0.5287104869578443, 0.5312154369068802, 0.5335319054808921, 0.5356095365453335, 0.537588246401803, 0.5394505342815815, 0.5411588929288605, 0.5427433552014741, 0.5442166523456129, 0.5455906131175243, 0.5469210900093435, 0.548091025874301, 0.5492619018641351, 0.5503253397566592, 0.5513288810881012, 0.5522800547874114, 0.5531239664595465, 0.5539805132437502, 0.5547984315965522, 0.5555853639099212, 
0.8, 0.75, 0.7, 0.6499999999999999, 0.5999999